In [1]:
 %%writefile Prac02a.cpp
#include <stdio.h>
#include <omp.h>
#include <time.h>

void bubbleSort(int arr[], int n) {
    int i, j;
    #pragma omp parallel for private(i, j) shared(arr)
    for (i = 0; i < n-1; i++) {
        // Last i elements are already in place, so only iterate till n-i-1
        for (j = 0; j < n-i-1; j++) {
            // Swap if the element found is greater than the next element
            if (arr[j] > arr[j+1]) {
                int temp = arr[j];
                arr[j] = arr[j+1];
                arr[j+1] = temp;
            }
        }
    }
}

int main() {
    int n;
    printf("Enter the number of elements: ");
    scanf("%d", &n);

    int arr[n];
    printf("Enter the elements: ");
    for (int i = 0; i < n; i++) {
        scanf("%d", &arr[i]);
    }

    printf("Unsorted array: \n");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    double start_time = omp_get_wtime();
    #pragma omp parallel
    {
        bubbleSort(arr, n);
    }
    double end_time = omp_get_wtime();

    printf("Sorted array: \n");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    printf("Total time taken by parallel section: %f seconds\n", end_time - start_time);
    printf("Number of threads used: %d\n", omp_get_max_threads());

    return 0;
}


Writing Prac02a.cpp


In [2]:
!g++ -fopenmp Prac02a.cpp -o Myexe

In [3]:
!./Myexe


Enter the number of elements: 5
Enter the elements: 10
27
90
0
22
Unsorted array: 
10 27 90 0 22 
Sorted array: 
0 10 22 27 90 
Total time taken by parallel section: 0.000240 seconds
Number of threads used: 2


In [4]:
 %%writefile Prac02b.cpp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <time.h>

void merge(int arr[], int l, int m, int r) {
    int i, j, k;
    int n1 = m - l + 1;
    int n2 = r - m;

    // Create temporary arrays
    int L[n1], R[n2];

    // Copy data to temporary arrays L[] and R[]
    for (i = 0; i < n1; i++)
        L[i] = arr[l + i];
    for (j = 0; j < n2; j++)
        R[j] = arr[m + 1 + j];

    // Merge the temporary arrays back into arr[l..r]
    i = 0; // Initial index of first subarray
    j = 0; // Initial index of second subarray
    k = l; // Initial index of merged subarray
    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            i++;
        } else {
            arr[k] = R[j];
            j++;
        }
        k++;
    }

    // Copy the remaining elements of L[], if any
    while (i < n1) {
        arr[k] = L[i];
        i++;
        k++;
    }

    // Copy the remaining elements of R[], if any
    while (j < n2) {
        arr[k] = R[j];
        j++;
        k++;
    }
}

void mergeSort(int arr[], int l, int r) {
    if (l < r) {
        // Same as (l+r)/2, but avoids overflow for large l and r
        int m = l + (r - l) / 2;

        // Sort first and second halves in parallel
        #pragma omp parallel sections
        {
            #pragma omp section
            {
                mergeSort(arr, l, m);
            }
            #pragma omp section
            {
                mergeSort(arr, m + 1, r);
            }
        }

        merge(arr, l, m, r);
    }
}

int main() {
    int n;
    printf("Enter the number of elements: ");
    scanf("%d", &n);

    int arr[n];
    printf("Enter the elements: ");
    for (int i = 0; i < n; i++) {
        scanf("%d", &arr[i]);
    }

    printf("Unsorted array: \n");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    double start_time = omp_get_wtime();
    // Perform merge sort
    #pragma omp parallel
    {
        #pragma omp single
        mergeSort(arr, 0, n - 1);
    }
    double end_time = omp_get_wtime();

    printf("Sorted array: \n");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    printf("Time taken: %f seconds\n", end_time - start_time);
    printf("Number of threads used: %d\n", omp_get_max_threads());

    return 0;
}


Writing Prac02b.cpp


In [5]:
!g++ -fopenmp Prac02b.cpp -o Myexe

In [6]:
!./Myexe


Enter the number of elements: 5
Enter the elements: 2 
7
88
55
21
Unsorted array: 
2 7 88 55 21 
Sorted array: 
2 7 21 55 88 
Time taken: 0.000138 seconds
Number of threads used: 2


In [7]:
# Let's analyze both parallel versions of the code individually and compare them to their sequential counterparts.

# ### Prac02a.cpp - Parallel Bubble Sort

# - **Parallelism**: In the parallel version, the bubble sort algorithm is executed concurrently by multiple threads using OpenMP parallelization.
# - **Performance Improvement**: The parallel version can potentially exhibit better performance compared to the sequential version for large datasets and systems with multiple cores. This is because parallel execution allows multiple threads to simultaneously work on different portions of the array, potentially reducing the overall execution time.
# - **Time Complexity**: Bubble sort has a time complexity of O(n^2) in both its sequential and parallel versions. Parallelizing bubble sort does not change its time complexity; however, it can improve its performance by distributing the workload across multiple threads.

# ### Prac02b.cpp - Parallel Merge Sort

# - **Parallelism**: In the parallel version, the merge sort algorithm is executed concurrently by multiple threads using OpenMP parallelization.
# - **Performance Improvement**: The parallel version can exhibit significantly better performance compared to the sequential version, especially for large datasets. Merge sort has a time complexity of O(n log n), which makes it more efficient than bubble sort. Parallelizing merge sort allows multiple threads to work on sorting different portions of the array simultaneously, resulting in faster sorting times.
# - **Time Complexity**: Merge sort has a time complexity of O(n log n), both in its sequential and parallel versions. Parallelizing merge sort does not change its time complexity; however, it can significantly improve its performance by leveraging multiple threads to divide and conquer the sorting process efficiently.

# In summary, the parallel versions of both bubble sort and merge sort can potentially exhibit better performance compared to their sequential counterparts, especially for large datasets. While parallelization does not change the time complexities of the algorithms themselves, it can distribute the workload across multiple threads, leading to faster execution times, particularly for algorithms like merge sort with more efficient time complexities.